##### 4_featureEngineering

In [1]:
import os
import pandas as pd
import numpy as np

In [5]:
aprHospFeat = pd.read_csv('dataFiles/compiledDischargesHospitalFeaturesCensus.csv', low_memory=False)

In [7]:
aprHospFeat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3584714 entries, 0 to 3584713
Data columns (total 41 columns):
Unnamed: 0                                              int64
Health Service Area                                     object
Facility Id                                             float64
Length of Stay                                          object
Type of Admission                                       object
APR DRG Code                                            float64
APR DRG Description                                     object
APR Severity of Illness Code                            float64
Payment Typology 1                                      object
Payment Typology 2                                      object
Payment Typology 3                                      object
Emergency Department Indicator                          object
Total Charges                                           float64
Total Costs                                             float64
y

##### Recoding the categorical vars into OneHot

##### Payment Types 1,2,3

In [132]:
aprHospFeat['Payment Typology 1'].unique()[:-1]

array(['Medicare', 'Medicaid', 'Blue Cross/Blue Shield',
       'Private Health Insurance', 'Self-Pay', 'Federal/State/Local/VA',
       'Miscellaneous/Other', 'Department of Corrections',
       'Managed Care, Unspecified', 'Unknown'], dtype=object)

In [133]:
for i in range(len(aprHospFeat['Payment Typology 1'].unique())-1):
    aprHospFeat.loc[(aprHospFeat['Payment Typology 1'] == aprHospFeat['Payment Typology 1'].unique()[i]) | 
                (aprHospFeat['Payment Typology 2'] == aprHospFeat['Payment Typology 1'].unique()[i]) |
                (aprHospFeat['Payment Typology 3'] == aprHospFeat['Payment Typology 1'].unique()[i]), 
                    aprHospFeat['Payment Typology 1'].unique()[i]] = 1

In [134]:
aprHospFeat[aprHospFeat['Payment Typology 1'].unique()[:-1]].fillna(0, inplace=True)

/nfshome/yg833/.conda/envs/mgPUI/lib/python3.7/site-packages/pandas/core/frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


##### Type of Admission

In [135]:
aprHospFeat['Type of Admission'].unique()

array(['Urgent', 'Elective', 'Emergency', 'Newborn', 'Not Available',
       'Trauma', nan], dtype=object)

In [136]:
aprHospFeat = pd.get_dummies(aprHospFeat, columns=['Type of Admission'])

##### Emergency Department Indicator

In [137]:
aprHospFeat['Emergency Department Indicator'].unique()

array(['Y', 'N', nan], dtype=object)

In [138]:
aprHospFeat.loc[aprHospFeat['Emergency Department Indicator'] == 'Y', 'emergencyRoom' ]= 1

In [139]:
aprHospFeat['emergencyRoom'].fillna(0, inplace=True)

##### Hospital Ownership

In [141]:
aprHospFeat = pd.get_dummies(aprHospFeat, columns=['Hospital Ownership'])

In [142]:
aprHospFeat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5077005 entries, 0 to 5077004
Data columns (total 63 columns):
Health Service Area                                               object
Facility Id                                                       float64
Length of Stay                                                    object
APR DRG Code                                                      float64
APR DRG Description                                               object
APR Severity of Illness Code                                      float64
Payment Typology 1                                                object
Payment Typology 2                                                object
Payment Typology 3                                                object
Emergency Department Indicator                                    object
Total Charges                                                     float64
Total Costs                                                       float64
yr    

In [124]:
aprHospFeat.drop(['Payment Typology 1', 'Payment Typology 2', 'Payment Typology 3'], 
                axis=1, inplace=True)